In [80]:
import numpy as np

In [81]:
from numpy import random

class Matrix :
    def __init__(self, rows, cols) :
        self.rows = rows
        self.cols = cols
        self.data = np.zeros(shape=(rows,cols))

    # Funções Diversas
    def randomize(self) :
        for row in range(0, self.rows) :
            for col in range(0, self.cols) :
                self.data[row][col] = random.rand(1)
            
    @staticmethod
    def array_to_matrix(arr) :
        matrix_result = Matrix(len(arr), 1)
        for row in range(0, matrix_result.rows) :
            matrix_result.data[row] = arr[row]
        return matrix_result

    # Operações Estáticas Matriz x Matriz
    @staticmethod
    def escalar_multiply(matrix_param, escalar) :
        matrix_result = Matrix(matrix_param.rows, matrix_param.cols)
        for row in range(0, matrix_result.rows) :
            for col in range(0, matrix_result.cols) :
                matrix_result.data[row][col] = matrix_param.data[row][col] * escalar
        return matrix_result
    
    @staticmethod
    def transpose(matrix_param) :
        matrix_result = Matrix(matrix_param.cols, matrix_param.rows)
        for row in range(0, matrix_result.rows) :
            for col in range(0, matrix_result.cols) :
                matrix_result.data[col][row] = matrix_param.data[row][col]
        return matrix_result
    
    @staticmethod
    def hadamard(matrix_a, matrix_b) :
        matrix_result = Matrix(matrix_a.rows, matrix_a.cols) 
        for row in range(0, matrix_result.rows) :
            for col in range(0, matrix_result.cols) :
                matrix_result.data[row][col] = matrix_a.data[row][col] * matrix_b.data[row][col]
        return matrix_result
    
    @staticmethod
    def add(matrix_a, matrix_b) :
        matrix_result = Matrix(matrix_a.rows, matrix_a.cols) 
        for row in range(0, matrix_result.rows) :
            for col in range(0, matrix_result.cols) :
                matrix_result.data[row][col] = matrix_a.data[row][col] + matrix_b.data[row][col]
        return matrix_result
    
    @staticmethod
    def subtract(matrix_a, matrix_b) :
        matrix_result = Matrix(matrix_b.rows, matrix_a.cols) 
        for row in range(0, matrix_result.rows) :
            for col in range(0, matrix_result.cols) :
                matrix_result.data[row][col] = matrix_a.data[row][col] - matrix_b.data[row][col]
        return matrix_result
    
    @staticmethod
    def multiply(matrix_a, matrix_b) :
        matrix_result = Matrix(matrix_a.rows, matrix_a.cols) 
        for row in range(0, matrix_result.rows) :
            for col in range(0, matrix_result.cols) :
                matrix_result.data[row][col] = matrix_a.data[row][col] * matrix_b.data[row][col]
        return matrix_result
    
    @staticmethod
    def map(matrix_param, func) :
        matrix_result = Matrix(matrix_param.rows, matrix_param.cols)
        for row in range(0, matrix_result.rows) :
            for col in range(0, matrix_result.cols) :
                matrix_result.data[row][col] = func(matrix_param.data[row][col])
        return matrix_result
    
    def map(self, func) :
        for row in range(0, self.rows) :
            for col in range(0, self.cols) :
                self.data[row][col] = func(self.data[row][col])

In [82]:
import math
def sigmoid(x) : 
    return 1/(1 + math.exp(-x))

def dsigmoid(x) :
    return x * (1-x)

class RNA : 
    def __init__(self, input_size, hidden_size,output_size, learning_rate = 0.1):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.bias_input_hidden = Matrix(self.hidden_size, 1)
        self.bias_input_hidden.randomize()
        self.bias_hidden_output = Matrix(self.output_size, 1)
        self.bias_hidden_output.randomize()
        
        self.weigths_input_hidden = Matrix(self.hidden_size, 1)
        self.weigths_input_hidden.randomize()
        self.weigths_hideen_output = Matrix(self.output_size, 1)
        self.weigths_hideen_output.randomize()
        self.learning_rate = learning_rate

    def train(self, arr, target) : 

        #Input -> Hidden
        input = Matrix.array_to_matrix(arr)
        print(self.weigths_input_hidden.data, input.data)
        hidden = Matrix.multiply(self.weigths_input_hidden, input)

        hidden = Matrix.add(hidden, self.bias_input_hidden)
        hidden.map(sigmoid)

        #Hidden -> Output
        output = Matrix.multiply(self.weigths_hideen_output, hidden)
        output = Matrix.add(output, self.bias_hidden_output)
        output.map(sigmoid)

        #Backpropagation
        expected = Matrix.array_to_matrix(target)
        expected = Matrix.arrayToMatrix(target)
        output_error = Matrix.subtract(expected,output)
        d_output = Matrix.map(output,dsigmoid)
        hidden_T = Matrix.transpose(hidden)

        gradient = Matrix.hadamard(d_output,output_error)
        gradient = Matrix.escalar_multiply(gradient,self.learning_rate)

        # Adjust Bias O->H
        self.bias_hidden_output = Matrix.add(self.bias_hidden_output, gradient)
        #Adjust Weigths O->H
        weigths_ho_deltas = Matrix.multiply(gradient,hidden_T)
        self.weigths_hideen_output = Matrix.add(self.weigths_hideen_output,weigths_ho_deltas)

        #Hidden -> Input
        weigths_ho_T = Matrix.transpose(self.bias_hidden_output)
        hidden_error = Matrix.multiply(weigths_ho_T,output_error)
        d_hidden = Matrix.map(hidden,dsigmoid)
        input_T = Matrix.transpose(input)

        gradient_H = Matrix.hadamard(d_hidden,hidden_error);
        gradient_H = Matrix.escalar_multiply(gradient_H, self.learning_rate);

        self.bias_input_hidden = Matrix.add(self.bias_input_hidden, gradient_H);
        # Adjust Weigths H->I
        weigths_ih_deltas = Matrix.multiply(gradient_H, input_T);
        self.weigths_ih = Matrix.add(self.bias_input_hidden, weigths_ih_deltas);

    def predict(self, arr) :
        input = Matrix.array_to_matrix(arr)
        hidden = Matrix.multiply(self.weigths_input_hidden, input)
        hidden = Matrix.add(hidden, self.bias_input_hidden)
        hidden.map(sigmoid)
        output = Matrix.multiply(self.weigths_hideen_output, hidden)
        output = Matrix.add(output, self.bias_hidden_output)
        output.map(sigmoid)
        output = Matrix.MatrixToArray(output);
        return output
    

In [83]:
nn = RNA(2,3,1)

inputs = [[1,1], [1,0], [0,1], [0,0]]
outputs = [[0], [1], [1], [0]]

for i in range (0, 10000) :
    index = random.randint(4)
    nn.train(inputs[index], outputs[index])
    


[[0.99098751]
 [0.82422571]
 [0.5836422 ]] [[0.]
 [0.]]


C:\Users\rafae\AppData\Local\Temp\ipykernel_7652\2655039789.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.data[row][col] = random.rand(1)


IndexError: index 2 is out of bounds for axis 0 with size 2